In [ ]:
#STS-ECG GitHub Issue 39 "Create MGH AVR features & labels CSV #39"

import pandas as pd

#Isolate subset of mgh-all-features-labels.csv using MRNs in mgh-avr.csv.

mghallfeatureslabels = pd.read_csv("mgh-all-features-labels.csv")
mghallfeatureslabels['medrecn_surgdt'] = mghallfeatureslabels.medrecn.astype(str) + "_" + mghallfeatureslabels.surgdt                                         

mghecgreadpcsupranodalrhythms = pd.read_csv("mgh-ecg-read-pc-supranodal-rhythms.csv")
mghecgreadpcsupranodalrhythms['ecg_patientid_clean_ecg_datetime'] = mghecgreadpcsupranodalrhythms.partners_ecg_patientid_clean.astype(str) + "_" + mghecgreadpcsupranodalrhythms.partners_ecg_datetime.astype(str)

#To find AFib labels in the 3-6 mo post-op range, cross-reference a) MRNs of patients with an ECG in 3-6 mo
#post-op range (dropbox:/sts-afib/results/sts-crossref-3-to-6-mo-postop/list_1+_in_all_windows.csv) against 
#MRNs of AFib labels: (dropbox:/sts-data/mgh-ecg-read-pc-supranodal-rhythms.csv). Description of label CSV here.

stscrossref3to6mopostop = pd.read_csv("list_1+_in_all_windows.csv")

In [ ]:
stscrossref3to6mopostop

In [ ]:
stscrossref3to6mopostop['surgdt_str'] = stscrossref3to6mopostop.surgdt.astype(str)
stscrossref3to6mopostop['surgdt_str'] = stscrossref3to6mopostop['surgdt_str'].str[:10]
stscrossref3to6mopostop['medrecn_surgdt'] = stscrossref3to6mopostop.medrecn.astype(str) + "_" + stscrossref3to6mopostop.surgdt_str
stscrossref3to6mopostop['ecg_patientid_clean_ecg_datetime'] = stscrossref3to6mopostop.partners_ecg_patientid_clean.astype(str) + "_" + stscrossref3to6mopostop.partners_ecg_datetime.astype(str)
stscrossref3to6mopostop['ecg_patientid_clean_ecg_datetime'] = stscrossref3to6mopostop['ecg_patientid_clean_ecg_datetime'].str.replace(' ','T') 
stscrossref3to6mopostop = stscrossref3to6mopostop.merge(mghecgreadpcsupranodalrhythms, on='ecg_patientid_clean_ecg_datetime', how='left') 

sts_afib_272 = stscrossref3to6mopostop.groupby(['medrecn_surgdt']).sum()
sts_afib_272.loc[sts_afib_272['partners_ecg_read_pc_afib'] == 0, 'long-term-afib'] = 0
sts_afib_272.loc[sts_afib_272['partners_ecg_read_pc_afib'] > 0, 'long-term-afib'] = 1

#For each patient (MRN) with an ECG in the 3-6 mo post-op window, you should be able to find the AFib label 
#for that (MRN, ECG_datetime) tuple.

sts_afib_272 = sts_afib_272.loc[:, 'long-term-afib']
sts_afib_272 = sts_afib_272.reset_index()
sts_afib_272.to_csv('sts_afib_272.csv', index = None, header=True)

In [ ]:
mgh_avr_features_labels = mghavr.merge(sts_afib_272, on='medrecn_surgdt', how='left')
mgh_avr_features_labels = mgh_avr_features_labels.dropna(subset=['long-term-afib'])
mgh_avr_features_labels = mgh_avr_features_labels.drop('medrecn_surgdt', axis=1)
mgh_avr_features_labels['long-term-afib'] = mgh_avr_features_labels['long-term-afib'].astype(int)
mgh_avr_features_labels = mgh_avr_features_labels[['medrecn', 'surgdt', 'mtopd', 'cnstrokp', 'crenfail', 'cpvntlng',
       'deepsterninf', 'reop', 'anymorbidity', 'los', 'slos', 'llos', 'long-term-afib', 'opcab',
       'opvalve', 'hdef', 'perfustm', 'xclamptm', 'creatlst', 'wbc',
       'platelets', 'hct', 'heightcm', 'weightkg', 'gender', 'age', 'diabetes',
       'dialysis', 'hypertn', 'cva', 'infendty', 'chrlungd', 'immsupp', 'pvd',
       'cvd', 'prcvint', 'prcab', 'prvalve', 'chf', 'carshock', 'resusc',
       'classnyh', 'medinotr', 'medster', 'numdisv', 'vdstena', 'vdstenm',
       'vdinsufa', 'vdinsufm', 'vdinsuft', 'status', 'incidenc', 'pocpci',
       'pocpciin', 'medgp', 'cvdtia', 'cvdpcarsurg', 'medadp5days',
       'ethnicity', 'racecaucasian', 'raceblack', 'raceasian', 'racenativeam',
       'raceothernativepacific']]
mgh_avr_features_labels.to_csv('mgh-avr-features-labels.csv', index = False, header=True)